In [1]:
import requests
import pandas as pd
import boto3

In [2]:
# Definir chave de acesso da API

API_KEY = "FRAN123" 
symbol = "AAPL"

In [3]:
# Criar script de teste de consumo da API

url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=compact&apikey={API_KEY}"

response = requests.get(url)
data = response.json()

In [4]:
# Validar retorno (JSON/CSV) e tratamento de erros

if "Time Series (Daily)" in data:
    ts_data = data["Time Series (Daily)"]
else:
    print("Erro ou limite de requisições:", data)
    ts_data = None

In [5]:
# Armazenar os dados crus em CSV no Data Lake

if ts_data:
    df = pd.DataFrame(ts_data).T  # transpor para ter datas nas linhas
    df.index = pd.to_datetime(df.index)  # index como datetime
    df = df.astype(float)  # converter valores para float
    print(df.head(50))

            1. open   2. high    3. low  4. close    5. volume
2025-09-12  229.220  234.5100  229.0200    234.07   55824216.0
2025-09-11  226.875  230.4500  226.6500    230.03   50208578.0
2025-09-10  232.185  232.4200  225.9500    226.79   83440810.0
2025-09-09  237.000  238.7805  233.3600    234.35   66313918.0
2025-09-08  239.300  240.1500  236.3400    237.88   48999495.0
2025-09-05  239.995  241.3200  238.4901    239.69   54870397.0
2025-09-04  238.450  239.8999  236.7400    239.78   47549429.0
2025-09-03  237.210  238.8500  234.3600    238.47   66061716.0
2025-09-02  229.250  230.8500  226.9700    229.72   44075638.0
2025-08-29  232.510  233.3800  231.3700    232.14   39418437.0
2025-08-28  230.820  233.4100  229.3350    232.56   38074700.0
2025-08-27  228.610  230.9000  228.2600    230.49   31259513.0
2025-08-26  226.870  229.4900  224.6900    229.31   54575107.0
2025-08-25  226.480  229.3000  226.2300    227.16   30983133.0
2025-08-22  226.170  229.0900  225.4100    227.76   424

In [ ]:
# Armazenar os dados crus em CSV no Data Lake
s3 = boto3.resource(
    "s3",
    endpoint_url="http://localhost:9000", 
    aws_access_key_id="admin",
    aws_secret_access_key="franadmin",
    region_name="us-east-1"
)

bucket_name = "raw-api-data"
arquivo_csv = f"raw_api_{symbol}_{pd.Timestamp.now().strftime('%Y%m%d%H%M%S')}.csv"
df.to_csv(arquivo_csv, index=True)

# Cria o bucket se não existir
if bucket_name not in [b.name for b in s3.buckets.all()]:
    s3.create_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' criado com sucesso.")

# Upload CSV para o bucket
s3.Bucket(bucket_name).upload_file(arquivo_csv, arquivo_csv)
print(f"Arquivo '{arquivo_csv}' enviado para o Data Lake.")

Bucket 'raw-api-data' criado com sucesso.
Arquivo 'raw_api_AAPL_20250913133210.csv' enviado para o Data Lake.


In [ ]:
# Documentar endpoint e formato dos dados
{
  "Time Series (Daily)": {
    "YYYY-MM-DD": {
      "1. open": "valor",
      "2. high": "valor",
      "3. low": "valor",
      "4. close": "valor",
      "5. volume": "valor"
    },
    ...
  }
}